<a href="https://colab.research.google.com/github/sl-gbf/counter-calculator/blob/main/Counter_RNG_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Counter RNG Calculator

Use this to *overestimate* the failure rate of your OTK, assuming you slightly overestimate the damage you take and your failure rate **only** depends on counter RNG.

This does not handle variance for the following other factors:
- multiattack
- nonconsistent critical mods
- RNG effects in general

To be specific, this code computes all possible targetting/dodge procs and checks if it means the necessary counter conditions.

To use this, copy this colab so you can save your own values for later.
Success rate is at the bottom after running Runtime -> Run All

In [ ]:
# Imports
from enum import Enum

# 4 frontline units
class Ch(Enum):
  A=0,
  B=1,
  C=2,
  D=3

hp = {}
dodge = {}
dmg_taken = {}
host = {}

# hp = 0 -> char DNE
# colabs forms doesn't support inputting it right into the dict, so it's a bit messy.
#@markdown Character A (Usually MC)
hp_A = 29990 #@param {type: "number", min:0}
dodge_A = 0.03 #@param {type:"number", min:0, max:1}
dmg_taken_A = 2450 #@param {type: "number", min:0}
hostility_A = 10 #@param {type:"number", min:0}
hp[Ch.A] = hp_A
dodge[Ch.A] = dodge_A
dmg_taken[Ch.A] = dmg_taken_A
host[Ch.A] = hostility_A

#@markdown Character B 

#@markdown For missing characters, set HP to 0.
hp_B = 24488 #@param {type: "number", min:0}
dodge_B = 0.00 #@param {type:"number", min:0, max:1}
dmg_taken_B = 2550 #@param {type: "number", min:0}
hostility_B = 10 #@param {type:"number", min:0}
hp[Ch.B] = hp_B
dodge[Ch.B] = dodge_B
dmg_taken[Ch.B] = dmg_taken_B
host[Ch.B] = hostility_B

#@markdown Character C 
hp_C = 27968 #@param {type: "number", min:0}
dodge_C = 0.00 #@param {type:"number", min:0, max:1}
dmg_taken_C = 4400 #@param {type: "number", min:0}
hostility_C = 10 #@param {type:"number", min:0}
hp[Ch.C] = hp_C
dodge[Ch.C] = dodge_C
dmg_taken[Ch.C] = dmg_taken_C
host[Ch.C] = hostility_C

#@markdown Character D
hp_D = 27684 #@param {type: "number", min:0}
dodge_D = 0.00 #@param {type:"number", min:0, max:1}
dmg_taken_D = 2750 #@param {type: "number", min:0}
hostility_D = 10 #@param {type:"number", min:0}
hp[Ch.D] = hp_D
dodge[Ch.D] = dodge_D
dmg_taken[Ch.D] = dmg_taken_D
host[Ch.D] = hostility_D


In [ ]:
#@markdown Select the configurations you want by commenting/uncommenting the appropiate lines:

#@markdown For example, `{Ch.A:1, Ch.B:1}` means that if Character A and B counters once (ie 1 hit), that's enough to guarantee an OTK

PASS_CONFIGURATIONS = [
    {Ch.A:1},
    #{Ch.B:1},
    #{Ch.C:1},
    #{Ch.D:1},
    #{Ch.A:1, Ch.B:1},
    #{Ch.A:1, Ch.C:1},
    #{Ch.A:1, Ch.D:1},
    {Ch.B:1, Ch.C:1},
    {Ch.B:1, Ch.D:1},
    {Ch.C:1, Ch.D:1},
    #{Ch.A:1, Ch.B:1, Ch.C:1},
    #{Ch.A:1, Ch.B:1, Ch.D:1},
    #{Ch.A:1, Ch.C:1, Ch.D:1},
    #{Ch.B:1, Ch.C:1, Ch.D:1},
    #{Ch.A:1, Ch.B:1, Ch.C:1, Ch.D:1},
]

In [ ]:
#@title
import math
STUPID_ORDERING = [Ch.A, Ch.B, Ch.C, Ch.D]
STUPID_INVERSE = {
    Ch.A:0, Ch.B:1, Ch.C:2, Ch.D:3
}
# uaw if you want to verify the probabilities adds up. Otherwise, adds useless computation.
DEBUG = False

def valid_hp_set(config, c, k):
  return (config[STUPID_INVERSE[c]] > 0 and config[STUPID_INVERSE[c]] <= (hp[c] - k*dmg_taken[c]))

def passing_configuration(config):
  if DEBUG:
    print("===== Cpnfiguration " + str(config) + " =====")
  for cond in PASS_CONFIGURATIONS:
    valid = True
    for ch in cond:
      if not valid_hp_set(config, ch, cond[ch]):
        valid = False
    if valid:
      if DEBUG:
        print("Pass due to condition set :" + str(cond))
        print("===== End of Config " + str(config) + " =====")
      return True
    if DEBUG:
      print("Violates all passing configs.")
      print("===== End of Config " + str(config) + " =====")
  return False

def generate_combinations():
  # we do this iteratively, as then we can compact the data representation every iteration
  old_tmp = [(tuple(hp[c] for c in STUPID_ORDERING), 1)]
  tmp = []
  for _ in range(10):
    for combo in old_tmp:
      conf=combo[0]
      prob=combo[1]
      total_host = sum([host[STUPID_ORDERING[c]] for c in range(len(STUPID_ORDERING)) if conf[c] > 0])
      if (total_host > 0):
        total_dodge_rate = sum([host[STUPID_ORDERING[c]]*dodge[STUPID_ORDERING[c]] for c in range(len(STUPID_ORDERING)) if conf[c] > 0]) / total_host
        if total_dodge_rate > 0:
          tmp.append((conf, prob*total_dodge_rate))
        for target in [c for c in range(len(STUPID_ORDERING)) if conf[c] > 0]:
          nct = list(conf)
          nct[target] -= dmg_taken[STUPID_ORDERING[target]]
          tmp.append((tuple(nct), prob*host[STUPID_ORDERING[target]]*(1-dodge[STUPID_ORDERING[target]])/total_host))
      else:
        # everyone has into hiding. some bigbrain leave it all to summer bika strat.... wait shit I want to try that.
        num_units = len([x for x in conf if x > 0])
        if num_units == 0:
          # this config is doomed, total party wipe, but we include this to guarantee total prob = 1 if debugging
          if DEBUG:
            tmp.append((conf, prob))
          continue
        total_dodge_rate = sum([dodge[STUPID_ORDERING[c]] for c in range(len(STUPID_ORDERING)) if conf[c] > 0]) / num_units
        if total_dodge_rate > 0:
          tmp.append((conf, prob*total_dodge_rate))
        for target in [c for c in range(len(STUPID_ORDERING)) if conf[c] > 0]:
          nct = list(conf)
          nct[target] -= dmg_taken[STUPID_ORDERING[target]]
          tmp.append((tuple(nct), prob*(1-dodge[STUPID_ORDERING[target]])/num_units))
    config_set = set(x[0] for x in tmp)
    old_tmp = [(x, sum([k[1] for k in tmp if x==k[0]])) for x in config_set]
    if DEBUG:
      print(">>>> Config Set: " + str(config_set))
      print("Resulting data: " + str(old_tmp))
      print("Config Probability Sum = " + str(sum([k[1] for k in old_tmp])))
      print(">>>>>>>>")
    tmp = []
  return old_tmp
    
def pass_rate():
  dat = generate_combinations()
  return sum([k[1] for k in dat if passing_configuration(k[0])])



print("Success Rate: " + str(pass_rate()))



Success Rate: 0.9997136785048484
